Analysing the IROS Poster Data, for
object      : cylinder
object pose : 0,0,0
controller  : 1,2,3
with and without interaction

1. Analyse Control Data
2. Visualise Behaviour Data
3. Preprocess control & behaviour data
4. Analyse Data with MC_W, MC_MI, and MC_CW


In [1]:
using PyPlot
using DataFrames
using ProgressMeter

/Users/zahedi/Library/Python/2.7/lib/python/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


### Functions & Variables

In [2]:
parent = "/Users/zahedi/projects/TU.Berlin/experiments/poster";
name(hand,suffix,controller) = "$parent/rbohand$hand$suffix/hand0_0-controller$controller-objectcylinder_0.0_0.0_0.0"
shortname(hand,suffix,controller) = "rbohand$hand$suffix/hand0_0-controller$controller-objectcylinder_0.0_0.0_0.0"
shortname(hand,suffix,controller,object) = "rbohand$hand$suffix/hand0_0-controller$controller-$(object)"

function get_xyz(filename)
    fd = open(filename)
    lines = readlines(fd)
    close(fd)
    lines = lines[2:2:end]
    s = split(strip(lines[1]), " ")
    v = collect(1:7:length(s)-1)
    v = [[i,i+1,i+2] for i in v]
    indices = v[1]
    for i = 2:length(v)
        indices = vcat(indices, v[i])
    end
    indices = indices .+ 1
    values = zeros(length(lines), length(indices))
    index = 1
    for l in lines
        s = split(strip(l), " ")
        values[index,:] = [parse(Float64,v) for v in s[indices]]
        index = index + 1
    end
    return values
end

function get_xyz(hand, suffix, controller)
    filename = "$(name(hand,suffix,controller))/raw/hand.sofastates.txt"
    return get_xyz(filename)
end

check_differences(a, b, column) = sqrt(sum((a[column] - b[column]).^2))

function get_value_by_name(df, name)
    return df[df[:Name] .== name, :]
end

roundF(v::Float64) = round(v * 1000.0) / 1000.0

roundF (generic function with 1 method)

In [3]:
function plotHand(data,line,xmin,xmax,ymin,ymax,zmin,zmax)
    f = figure()
    xlim([xmin,xmax])
    ylim([ymin,ymax])
    zlim([zmin,zmax])
    plot3D(
    [data[line,1+3],data[line,1+2*3]],
    [data[line,2+3],data[line,2+2*3]],
    [data[line,3+3],data[line,3+2*3]]
    )

    for i in collect(1:5)
        plot3D(
        [data[line,1+i*3],data[line,1+(i+1)*3]],
        [data[line,2+i*3],data[line,2+(i+1)*3]],
        [data[line,3+i*3],data[line,3+(i+1)*3]])
    end
    # -> 6 is fingertip 1
    
    xlim([xmin,xmax])
    ylim([ymin,ymax])
    zlim([zmin,zmax])
    plot3D(
    [data[line,1+3],data[line,1+7*3]],
    [data[line,2+3],data[line,2+7*3]],
    [data[line,3+3],data[line,3+7*3]]
    )

    for i in collect(7:10)
        plot3D(
        [data[line,1+i*3],data[line,1+(i+1)*3]],
        [data[line,2+i*3],data[line,2+(i+1)*3]],
        [data[line,3+i*3],data[line,3+(i+1)*3]])
    end
    # -> 11 is fingertip 2

    plot3D(
    [data[line,1+3],data[line,1+12*3]],
    [data[line,2+3],data[line,2+12*3]],
    [data[line,3+3],data[line,3+12*3]]
    )

    for i in collect(12:15)
        plot3D(
        [data[line,1+i*3],data[line,1+(i+1)*3]],
        [data[line,2+i*3],data[line,2+(i+1)*3]],
        [data[line,3+i*3],data[line,3+(i+1)*3]])
    end
    # -> 16 is fingertip 3

    plot3D(
    [data[line,1+3],data[line,1+17*3]],
    [data[line,2+3],data[line,2+17*3]],
    [data[line,3+3],data[line,3+17*3]]
    )

    for i in collect(17:20)
        plot3D(
        [data[line,1+i*3],data[line,1+(i+1)*3]],
        [data[line,2+i*3],data[line,2+(i+1)*3]],
        [data[line,3+i*3],data[line,3+(i+1)*3]])
    end
    # -> 21 is fingertip 4

    plot3D(
    [data[line,1+3],data[line,1+22*3]],
    [data[line,2+3],data[line,2+22*3]],
    [data[line,3+3],data[line,3+22*3]]
    )
    # -> 22 is hand palm

    for i in collect(22:30)
        plot3D(
        [data[line,1+i*3],data[line,1+(i+1)*3]],
        [data[line,2+i*3],data[line,2+(i+1)*3]],
        [data[line,3+i*3],data[line,3+(i+1)*3]])
    end
    
    savefig("/tmp/plot_$line.png")
    close(f)
end

function plotHandRotated(data,line,xmin,xmax,ymin,ymax,zmin,zmax, e, a)
    f = figure()
    ax = Axes3D(f)
    ax[:view_init](elev=e, azim=a)
    
    xlim([xmin,xmax])
    ylim([ymin,ymax])
    zlim([zmin,zmax])
    plot3D(
    [data[line,1+3],data[line,1+2*3]],
    [data[line,2+3],data[line,2+2*3]],
    [data[line,3+3],data[line,3+2*3]]
    )

    for i in collect(1:5)
        plot3D(
        [data[line,1+i*3],data[line,1+(i+1)*3]],
        [data[line,2+i*3],data[line,2+(i+1)*3]],
        [data[line,3+i*3],data[line,3+(i+1)*3]])
    end
    # -> 6 is fingertip 1
    
    xlim([xmin,xmax])
    ylim([ymin,ymax])
    zlim([zmin,zmax])
    plot3D(
    [data[line,1+3],data[line,1+7*3]],
    [data[line,2+3],data[line,2+7*3]],
    [data[line,3+3],data[line,3+7*3]]
    )

    for i in collect(7:10)
        plot3D(
        [data[line,1+i*3],data[line,1+(i+1)*3]],
        [data[line,2+i*3],data[line,2+(i+1)*3]],
        [data[line,3+i*3],data[line,3+(i+1)*3]])
    end
    # -> 11 is fingertip 2

    plot3D(
    [data[line,1+3],data[line,1+12*3]],
    [data[line,2+3],data[line,2+12*3]],
    [data[line,3+3],data[line,3+12*3]]
    )

    for i in collect(12:15)
        plot3D(
        [data[line,1+i*3],data[line,1+(i+1)*3]],
        [data[line,2+i*3],data[line,2+(i+1)*3]],
        [data[line,3+i*3],data[line,3+(i+1)*3]])
    end
    # -> 16 is fingertip 3

    plot3D(
    [data[line,1+3],data[line,1+17*3]],
    [data[line,2+3],data[line,2+17*3]],
    [data[line,3+3],data[line,3+17*3]]
    )

    for i in collect(17:20)
        plot3D(
        [data[line,1+i*3],data[line,1+(i+1)*3]],
        [data[line,2+i*3],data[line,2+(i+1)*3]],
        [data[line,3+i*3],data[line,3+(i+1)*3]])
    end
    # -> 21 is fingertip 4

    plot3D(
    [data[line,1+3],data[line,1+22*3]],
    [data[line,2+3],data[line,2+22*3]],
    [data[line,3+3],data[line,3+22*3]]
    )

    for i in collect(22:30)
        plot3D(
        [data[line,1+i*3],data[line,1+(i+1)*3]],
        [data[line,2+i*3],data[line,2+(i+1)*3]],
        [data[line,3+i*3],data[line,3+(i+1)*3]])
    end
    
    
    savefig("/tmp/plot_$line.png")
    close(f)
end

# data = get_xyz(2,"-prescriptive",0)
# xmin = minimum([minimum(data[:,i]) for i in collect(1:3:size(data)[2])])
# xmax = maximum([maximum(data[:,i]) for i in collect(1:3:size(data)[2])])

# ymin = minimum([minimum(data[:,i]) for i in collect(2:3:size(data)[2])])
# ymax = maximum([maximum(data[:,i]) for i in collect(2:3:size(data)[2])])

# zmin = minimum([minimum(data[:,i]) for i in collect(3:3:size(data)[2])])
# zmax = maximum([maximum(data[:,i]) for i in collect(3:3:size(data)[2])])

#plotHandRotated(data,10,xmin,xmax,ymin,ymax,zmin,zmax, 0, 0)
#plotHandRotated(data,10,xmin,xmax,ymin,ymax,zmin,zmax, -40, 90)

plotHandRotated (generic function with 1 method)

In [4]:
function create_video(hand,suffix,controller)
    data = get_xyz(hand,suffix,controller)

    xmin = minimum([minimum(data[:,i]) for i in collect(1:3:size(data)[2])])
    xmax = maximum([maximum(data[:,i]) for i in collect(1:3:size(data)[2])])

    ymin = minimum([minimum(data[:,i]) for i in collect(2:3:size(data)[2])])
    ymax = maximum([maximum(data[:,i]) for i in collect(2:3:size(data)[2])])

    zmin = minimum([minimum(data[:,i]) for i in collect(3:3:size(data)[2])])
    zmax = maximum([maximum(data[:,i]) for i in collect(3:3:size(data)[2])])

    for i in 1:size(data)[1]
        plotHand(data,i,xmin,xmax,ymin,ymax,zmin,zmax)
    end
    
    name = shortname(hand,suffix,controller)
    name = replace(name,"/","_")

    run(`ffmpeg -y -framerate 5 -i /tmp/plot_%d.png -c:v libx264 -r 30 -pix_fmt yuv420p /Users/zahedi/Desktop/$name.mp4`)

    for i in 1:size(data)[1]
        rm("/tmp/plot_$i.png")
    end
end

function create_video_rotated(hand,suffix,controller, e, a)
    data = get_xyz(hand,suffix,controller)

    xmin = minimum([minimum(data[:,i]) for i in collect(1:3:size(data)[2])])
    xmax = maximum([maximum(data[:,i]) for i in collect(1:3:size(data)[2])])

    ymin = minimum([minimum(data[:,i]) for i in collect(2:3:size(data)[2])])
    ymax = maximum([maximum(data[:,i]) for i in collect(2:3:size(data)[2])])

    zmin = minimum([minimum(data[:,i]) for i in collect(3:3:size(data)[2])])
    zmax = maximum([maximum(data[:,i]) for i in collect(3:3:size(data)[2])])

    for i in 1:size(data)[1]
        plotHandRotated(data,i,xmin,xmax,ymin,ymax,zmin,zmax, e, a)
    end
    
    name = shortname(hand,suffix,controller)
    name = replace(name,"/","_")

    run(`ffmpeg -y -framerate 5 -i /tmp/plot_%d.png -c:v libx264 -r 30 -pix_fmt yuv420p /Users/zahedi/Desktop/$name.mp4`)

    for i in 1:size(data)[1]
        rm("/tmp/plot_$i.png")
    end
end

function create_video_rotated(experiment, e, a)
    data = get_xyz("$parent/$experiment/raw/hand.sofastates.txt")

    xmin = minimum([minimum(data[:,i]) for i in collect(1:3:size(data)[2])])
    xmax = maximum([maximum(data[:,i]) for i in collect(1:3:size(data)[2])])

    ymin = minimum([minimum(data[:,i]) for i in collect(2:3:size(data)[2])])
    ymax = maximum([maximum(data[:,i]) for i in collect(2:3:size(data)[2])])

    zmin = minimum([minimum(data[:,i]) for i in collect(3:3:size(data)[2])])
    zmax = maximum([maximum(data[:,i]) for i in collect(3:3:size(data)[2])])

    for i in 1:size(data)[1]
        plotHandRotated(data,i,xmin,xmax,ymin,ymax,zmin,zmax, e, a)
    end
    
    name = experiment
    name = replace(name,"/","_")

    run(`ffmpeg -y -framerate 5 -i /tmp/plot_%d.png -c:v libx264 -r 30 -pix_fmt yuv420p /Users/zahedi/Desktop/$name.mp4`)

    for i in 1:size(data)[1]
        rm("/tmp/plot_$i.png")
    end
end

# create_video_rotated(2,"",0,-40, 90)

# create_video_rotated("rbohand2-prescriptive/hand0_0-controller0-objecteggB_0.0_0.0_0.0", -40, 90)

ffmpeg version 3.1.4 Copyright (c) 2000-2016 the FFmpeg developers
  built with Apple LLVM version 8.0.0 (clang-800.0.38)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/3.1.4 --enable-shared --enable-pthreads --enable-gpl --enable-version3 --enable-hardcoded-tables --enable-avresample --cc=clang --host-cflags= --host-ldflags= --enable-opencl --enable-libx264 --enable-libmp3lame --enable-libxvid --disable-lzma --enable-vda
  libavutil      55. 28.100 / 55. 28.100
  libavcodec     57. 48.101 / 57. 48.101
  libavformat    57. 41.100 / 57. 41.100
  libavdevice    57.  0.101 / 57.  0.101
  libavfilter     6. 47.100 /  6. 47.100
  libavresample   3.  0.  0 /  3.  0.  0
  libswscale      4.  1.100 /  4.  1.100
  libswresample   2.  1.100 /  2.  1.100
  libpostproc    54.  0.100 / 54.  0.100
Input #0, image2, from '/tmp/plot_%d.png':
  Duration: 00:00:27.80, start: 0.000000, bitrate: N/A
    Stream #0:0: Video: png, rgba(pc), 800x600 [SAR 3937:3937 DAR 4:3], 5 fps, 5 tbr, 5 tbn, 5 tbc
[lib

# 1. Analyse Control Data

## Reading all control data

In [5]:
rbohand2_controller_0_control_states = readtable("$(name(2,"",0))/raw/control.states.csv")
rbohand2_prescriptive_controller_0_control_states = readtable("$(name(2,"-prescriptive",0))/raw/control.states.csv")
rbohand3_controller_0_control_states = readtable("$(name(3,"",0))/raw/control.states.csv")
rbohand3_prescriptive_controller_0_control_states = readtable("$(name(3,"-prescriptive",0))/raw/control.states.csv")

rbohand2_controller_1_control_states = readtable("$(name(2,"",1))/raw/control.states.csv")
rbohand2_prescriptive_controller_1_control_states = readtable("$(name(2,"-prescriptive",1))/raw/control.states.csv")
rbohand3_controller_1_control_states = readtable("$(name(3,"",1))/raw/control.states.csv")
rbohand3_prescriptive_controller_1_control_states = readtable("$(name(3,"-prescriptive",1))/raw/control.states.csv")

rbohand2_controller_2_control_states = readtable("$(name(2,"",2))/raw/control.states.csv")
rbohand2_prescriptive_controller_2_control_states = readtable("$(name(2,"-prescriptive",2))/raw/control.states.csv")
rbohand3_controller_2_control_states = readtable("$(name(3,"",2))/raw/control.states.csv")
rbohand3_prescriptive_controller_2_control_states = readtable("$(name(3,"-prescriptive",2))/raw/control.states.csv")

,time,wrist_x,wrist_y,wrist_z,wrist_rx,wrist_ry,wrist_rz,wrist_rw,pressure_0,pressure_1,pressure_2,pressure_3,pressure_4,pressure_5
1,0.01,0.0,200.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,499.99999999999994
2,0.060000000000000005,0.0,200.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,3000.0
3,0.10999999999999999,0.0,197.50000000000003,0.0,-0.003235238063781503,0.0004259271713866455,-0.003235238063781503,0.9995740728286134,249.99999999999952,249.99999999999952,249.99999999999952,249.99999999999952,0.0,5000.0
4,0.16,0.0,185.0,0.0,-0.019411428382689055,0.0025555630283198775,-0.019411428382689055,0.9974444369716801,1500.0,1500.0,1500.0,1500.0,0.0,5000.0
5,0.20000000000000004,0.0,175.0,0.0,-0.0323523806378151,0.004259271713866464,-0.0323523806378151,0.9957407282861336,2500.0000000000005,2500.0000000000005,2500.0000000000005,2500.0000000000005,0.0,5000.0
6,0.25000000000000006,0.0,162.5,0.0,-0.04852857095672265,0.006388907570799696,-0.04852857095672265,0.9936110924292003,3750.000000000001,3750.000000000001,3750.000000000001,3750.000000000001,0.0,5000.0
7,0.3000000000000001,0.0,149.99999999999997,0.0,-0.06470476127563021,0.008518543427732929,-0.06470476127563021,0.991481456572267,5000.000000000002,5000.000000000002,5000.000000000002,5000.000000000002,0.0,5000.0
8,0.35000000000000014,0.0,137.49999999999997,0.0,-0.08088095159453776,0.010648179284666161,-0.08088095159453776,0.9893518207153338,6250.000000000002,6250.000000000002,6250.000000000002,6250.000000000002,0.0,5000.0
9,0.4000000000000002,0.0,124.99999999999997,0.0,-0.09705714191344532,0.012777815141599393,-0.09705714191344532,0.9872221848584005,7500.000000000004,7500.000000000004,7500.000000000004,7500.000000000004,0.0,5000.0
10,0.45000000000000023,0.0,112.49999999999996,0.0,-0.11323333223235288,0.014907450998532627,-0.11323333223235288,0.9850925490014674,8750.000000000004,8750.000000000004,8750.000000000004,8750.000000000004,0.0,5000.0


In [6]:
names1  = []
names2  = []
columns = []
l2      = []

for controller in [0,1,2]
    for hand1 in [2,3]
        for hand2 in [2,3]
            for column in [:wrist_x, :wrist_y, :wrist_z, :wrist_rx, :wrist_ry, :wrist_rz, :wrist_rw, :pressure_0, :pressure_1, :pressure_2, :pressure_3, :pressure_4, :pressure_5]
                name1 = name(hand1, "", controller)
                name2 = name(hand2, "-prescriptive", controller)
                if name1 != name2
                    a = readtable("$(name(hand1,"",controller))/raw/control.states.csv")
                    b = readtable("$(name(hand2,"-prescriptive",controller))/raw/control.states.csv")
                    append!(names1,  [shortname(hand1, "", controller)])
                    append!(names2,  [shortname(hand2, "-prescriptive", controller)])
                    append!(columns, [column])
                    append!(l2,[check_differences(a,b,column)])
                end
            end
        end
    end
end

for controller1 in [0,1,2]
    for controller2 in [0,1,2]
        for hand in [2,3]
            for column in [:wrist_x, :wrist_y, :wrist_z, :wrist_rx, :wrist_ry, :wrist_rz, :wrist_rw, :pressure_0, :pressure_1, :pressure_2, :pressure_3, :pressure_4, :pressure_5]
                name1 = name(hand, "", controller1)
                name2 = name(hand, "", controller2)
                if name1 != name2
                    a = readtable("$(name(hand,"",controller1))/raw/control.states.csv")
                    b = readtable("$(name(hand,"",controller2))/raw/control.states.csv")
                    append!(names1,  [shortname(hand, "", controller1)])
                    append!(names2,  [shortname(hand, "", controller2)])
                    append!(columns, [column])
                    append!(l2,[check_differences(a,b,column)])
                end
            end
        end
    end
end

differences_table = DataFrame()
differences_table[:First]  = names1
differences_table[:Second] = names2
differences_table[:Column] = columns
differences_table[:L2Norm] = l2
writetable("/Users/zahedi/Desktop/comparison.csv", differences_table)
differences_table

,First,Second,Column,L2Norm
1,rbohand2/hand0_0-controller0-objectcylinder_0.0_0.0_0.0,rbohand2-prescriptive/hand0_0-controller0-objectcylinder_0.0_0.0_0.0,wrist_x,0.0
2,rbohand2/hand0_0-controller0-objectcylinder_0.0_0.0_0.0,rbohand2-prescriptive/hand0_0-controller0-objectcylinder_0.0_0.0_0.0,wrist_y,0.0
3,rbohand2/hand0_0-controller0-objectcylinder_0.0_0.0_0.0,rbohand2-prescriptive/hand0_0-controller0-objectcylinder_0.0_0.0_0.0,wrist_z,0.0
4,rbohand2/hand0_0-controller0-objectcylinder_0.0_0.0_0.0,rbohand2-prescriptive/hand0_0-controller0-objectcylinder_0.0_0.0_0.0,wrist_rx,0.0
5,rbohand2/hand0_0-controller0-objectcylinder_0.0_0.0_0.0,rbohand2-prescriptive/hand0_0-controller0-objectcylinder_0.0_0.0_0.0,wrist_ry,0.0
6,rbohand2/hand0_0-controller0-objectcylinder_0.0_0.0_0.0,rbohand2-prescriptive/hand0_0-controller0-objectcylinder_0.0_0.0_0.0,wrist_rz,0.0
7,rbohand2/hand0_0-controller0-objectcylinder_0.0_0.0_0.0,rbohand2-prescriptive/hand0_0-controller0-objectcylinder_0.0_0.0_0.0,wrist_rw,0.0
8,rbohand2/hand0_0-controller0-objectcylinder_0.0_0.0_0.0,rbohand2-prescriptive/hand0_0-controller0-objectcylinder_0.0_0.0_0.0,pressure_0,0.0
9,rbohand2/hand0_0-controller0-objectcylinder_0.0_0.0_0.0,rbohand2-prescriptive/hand0_0-controller0-objectcylinder_0.0_0.0_0.0,pressure_1,0.0
10,rbohand2/hand0_0-controller0-objectcylinder_0.0_0.0_0.0,rbohand2-prescriptive/hand0_0-controller0-objectcylinder_0.0_0.0_0.0,pressure_2,0.0


# 2. Analyse Behaviour Data

In [7]:
rbohand2_prescriptive_controller_0_control_states = readtable("$(name(2,"-prescriptive",0))/raw/control.states.csv")

,time,wrist_x,wrist_y,wrist_z,wrist_rx,wrist_ry,wrist_rz,wrist_rw,pressure_0,pressure_1,pressure_2,pressure_3,pressure_4,pressure_5
1,0.01,0.0,200.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,499.99999999999994
2,0.060000000000000005,0.0,200.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,3000.0
3,0.10999999999999999,0.0,197.50000000000003,0.0,0.0,0.0,-0.003263154805501283,0.9997861215343453,249.99999999999952,249.99999999999952,249.99999999999952,249.99999999999952,0.0,5000.0
4,0.16,0.0,185.0,0.0,0.0,0.0,-0.019578928833007735,0.9987167292060715,1500.0,1500.0,1500.0,1500.0,0.0,5000.0
5,0.20000000000000004,0.0,175.0,0.0,0.0,0.0,-0.0326315480550129,0.9978612153434526,2500.0000000000005,2500.0000000000005,2500.0000000000005,2500.0000000000005,0.0,5000.0
6,0.25000000000000006,0.0,162.5,0.0,0.0,0.0,-0.048947322082519354,0.9967918230151789,3750.000000000001,3750.000000000001,3750.000000000001,3750.000000000001,0.0,5000.0
7,0.3000000000000001,0.0,149.99999999999997,0.0,0.0,0.0,-0.06526309611002581,0.9957224306869052,5000.000000000002,5000.000000000002,5000.000000000002,5000.000000000002,0.0,5000.0
8,0.35000000000000014,0.0,137.49999999999997,0.0,0.0,0.0,-0.08157887013753226,0.9946530383586315,6250.000000000002,6250.000000000002,6250.000000000002,6250.000000000002,0.0,5000.0
9,0.4000000000000002,0.0,124.99999999999997,0.0,0.0,0.0,-0.09789464416503872,0.9935836460303578,7500.000000000004,7500.000000000004,7500.000000000004,7500.000000000004,0.0,5000.0
10,0.45000000000000023,0.0,112.49999999999996,0.0,0.0,0.0,-0.11421041819254518,0.9925142537020841,8750.000000000004,8750.000000000004,8750.000000000004,8750.000000000004,0.0,5000.0


In [8]:
# for hand in [2,3]
#     for suffix in ["", "-prescriptive"]
#         for controller in [0,1,2]
#             println("working on $(shortname(hand, suffix, controller))")
#             create_video_rotated(hand, suffix, controller,-40,90)
#         end
#     end
# end

# 3. Preprocess control and behaviour data

In [9]:
parent = "/Users/zahedi/projects/TU.Berlin/experiments/poster";
subdir = readdir(parent)
subdir = filter(c->contains(c,"rbohand"),subdir)

dirs = []

for dir in subdir
    d = readdir("$parent/$dir")
    d = filter(c->contains(c,"hand"), d)
    d = ["$parent/$dir/$v" for v in d]
    append!(dirs, d)
end

dirs[1:10]

10-element Array{Any,1}:
 "/Users/zahedi/projects/TU.Berlin/experiments/poster/rbohand2/hand0_0-controller0-objectboxB_-10.0_-10.0_-15.0"
 "/Users/zahedi/projects/TU.Berlin/experiments/poster/rbohand2/hand0_0-controller0-objectboxB_-10.0_-10.0_-7.5" 
 "/Users/zahedi/projects/TU.Berlin/experiments/poster/rbohand2/hand0_0-controller0-objectboxB_-10.0_-10.0_0.0"  
 "/Users/zahedi/projects/TU.Berlin/experiments/poster/rbohand2/hand0_0-controller0-objectboxB_-10.0_-10.0_15.0" 
 "/Users/zahedi/projects/TU.Berlin/experiments/poster/rbohand2/hand0_0-controller0-objectboxB_-10.0_-10.0_7.5"  
 "/Users/zahedi/projects/TU.Berlin/experiments/poster/rbohand2/hand0_0-controller0-objectboxB_-10.0_-5.0_-15.0" 
 "/Users/zahedi/projects/TU.Berlin/experiments/poster/rbohand2/hand0_0-controller0-objectboxB_-10.0_-5.0_-7.5"  
 "/Users/zahedi/projects/TU.Berlin/experiments/poster/rbohand2/hand0_0-controller0-objectboxB_-10.0_-5.0_0.0"   
 "/Users/zahedi/projects/TU.Berlin/experiments/poster/rbohand2/hand0_0-

### 1) Preprocessing the files

In [10]:
function copy_and_convert_hand(dirs)
    in_dirs  = map(x->"$x/raw", dirs)
    out_dirs = map(x->"$x/analysis", dirs)
    
    p = Progress(length(in_dirs)*1, 1, "hand.sofastates.csv ...")

    for index in 1:length(in_dirs)
        if isfile("$(in_dirs[index])/hand.sofastates.txt")
            m = get_xyz("$(in_dirs[index])/hand.sofastates.txt")
            for i = 5:3:size(m)[2]
                m[:,i] = m[:,i] - m[:,2]
            end
            m = m[:,4:end]
            if isdir(out_dirs[index]) == false
                mkdir(out_dirs[index])
            end
            writecsv("$(out_dirs[index])/hand.sofastates.csv",m)
        end
        next!(p)
    end
end

copy_and_convert_hand (generic function with 1 method)

In [11]:
function copy_and_convert_object(dirs)
    in_dirs  = map(x->"$x/raw", dirs)
    out_dirs = map(x->"$x/analysis", dirs)
    
    p = Progress(length(in_dirs)*1, 1, "obstacle.sofastates.csv ...")

    for index in 1:length(in_dirs)
        if isfile("$(in_dirs[index])/obstacle.sofastates.txt")
            m = get_xyz("$(in_dirs[index])/obstacle.sofastates.txt")
            if isdir(out_dirs[index]) == false
                mkdir(out_dirs[index])
            end
            writecsv("$(out_dirs[index])/obstacle.sofastates.csv",m)
        end
        next!(p)
    end
end
#copy_and_convert_object(dirs)

copy_and_convert_object (generic function with 1 method)

In [12]:
function copy_and_convert_control(dirs)
    
    in_dirs  = map(x->"$x/raw", dirs)
    out_dirs = map(x->"$x/analysis", dirs)
    
    p = Progress(length(in_dirs), 1, "control.states.csv ...")

    for i in 1:length(in_dirs)
        if isfile("$(in_dirs[i])/control.states.csv")
            a = readcsv("$(in_dirs[i])/control.states.csv")[2:end-1,:]
            a = map(x->Float64(x),a)
            writecsv("$(out_dirs[i])/control.states.csv",a)
        end
        next!(p)
    end
end

function copy_and_convert_files(path)
    copy_and_convert_hand(path)
    copy_and_convert_object(path)
    copy_and_convert_control(path)
end

copy_and_convert_files (generic function with 1 method)

In [165]:
copy_and_convert_files(dirs)

hand.sofastates.csv ...100% Time: 0:03:02
obstacle.sofastates.csv ...100% Time: 0:00:59
control.states.csv ...100% Time: 0:03:40


### 2)  Get Min/Max Values

In [16]:
function write_domains(dirs)    
    in_dirs      = map(x->"$x/analysis", dirs)
    controlfiles = map(x->"$x/control.states.csv",in_dirs)
    a_domains    = map(x->"$x/A.domains.csv",in_dirs)
    handfiles    = map(x->"$x/hand.sofastates.csv",in_dirs)
    w_domains    = map(x->"$x/W.domains.csv",in_dirs)

    function minmax(in_,out_)
        a = readcsv(in_)
        b = minimum(a,1)
        c = maximum(a,1)
        d = vcat(b,c)
        writecsv(out_, d)
    end

    p = Progress(length(in_dirs), 1, "domain files ...")

    for i = 1:length(in_dirs)
        if isfile(controlfiles[i])
            minmax(controlfiles[i], a_domains[i])
        end
        if isfile(handfiles[i])
            minmax(handfiles[i],w_domains[i])
        end
        next!(p)
    end
end

write_domains(dirs)

domain files ...100% Time: 0:01:07


In [17]:
w_indices = join(["$(i*3+1),$(i*3+2),$(i*3+3)" for i in [5,10,15,20]],",")

"16,17,18,31,32,33,46,47,48,61,62,63"

### 3)  Hand, Controller, Objects, Positions

In [15]:
wbins     = 100
abins     = 30

# index 1 was removed when copying (wrist position)
# -> 5 is fingertip 1
# -> 10 is fingertip 2
# -> 15 is fingertip 3
# -> 20 is fingertip 4

w_indices = join(["$(i*3+1),$(i*3+2),$(i*3+3)" for i in [5,10,15,20]],",")
a_indices = "8,9,10,11"

in_dirs = map(x->"$x/analysis", dirs)

fd = open("Makefile.1","w")
write(fd,"all:\n")
for d in in_dirs
    command = "/Users/zahedi/projects/builds/entropy-ninja/bin/rbo_mc -wi $(w_indices) -ai $(a_indices) -wbins $wbins -abins $abins -d \"$d\""   
    write(fd,"\t$command\n")
end
close(fd)
run(`make -j 4 -f Makefile.1`)

/Users/zahedi/projects/builds/entropy-ninja/bin/rbo_mc -wi 16,17,18,31,32,33,46,47,48,61,62,63 -ai 8,9,10,11 -wbins 100 -abins 30 -d "/Users/zahedi/projects/TU.Berlin/experiments/poster/rbohand2/hand0_0-controller0-objectboxB_-10.0_-10.0_-15.0/analysis"
MC_W:  3.06602
MC_CW: 3.06602
/Users/zahedi/projects/builds/entropy-ninja/bin/rbo_mc -wi 16,17,18,31,32,33,46,47,48,61,62,63 -ai 8,9,10,11 -wbins 100 -abins 30 -d "/Users/zahedi/projects/TU.Berlin/experiments/poster/rbohand2/hand0_0-controller0-objectboxB_-10.0_-10.0_-7.5/analysis"
MC_W:  4.10268
MC_CW: 4.10268
/Users/zahedi/projects/builds/entropy-ninja/bin/rbo_mc -wi 16,17,18,31,32,33,46,47,48,61,62,63 -ai 8,9,10,11 -wbins 100 -abins 30 -d "/Users/zahedi/projects/TU.Berlin/experiments/poster/rbohand2/hand0_0-controller0-objectboxB_-10.0_-10.0_0.0/analysis"
MC_W:  4.26212
MC_CW: 4.26212
/Users/zahedi/projects/builds/entropy-ninja/bin/rbo_mc -wi 16,17,18,31,32,33,46,47,48,61,62,63 -ai 8,9,10,11 -wbins 100 -abins 30 -d "/Users/zahedi/pro

### Auswertung

In [18]:
mcw_files  = map(x->"$x/analysis/mc_w-averaged_$(wbins)_$(abins).txt", dirs)
mcw_values = map(x->Float64(readcsv(x)[1]),mcw_files)
names      = []
fd = open("$parent/results_mc_w-averaged_$(wbins)_$(abins).txt","w")
for i = 1:length(in_dirs)
    v = split(in_dirs[i],"/")
    n = "$(v[8])/$(v[9])"
    append!(names,[n])
    write(fd, "$n,$(mcw_values[i])\n")
end
close(fd)

mcw_results = DataFrame()
mcw_results[:Name] = names
mcw_results[:Values] = mcw_values
sort!(mcw_results, cols=[:Values])
mcw_results[:,:Values] = map(Float64,mcw_results[:,:Values])
writetable("$parent/results_mc_w-averaged_$(wbins)_$(abins).csv",mcw_results)
mcw_results

,Name,Values
1,rbohand2/hand0_0-controller0-objectbox_10.0_-10.0_-15.0,3.0417
2,rbohand2/hand0_0-controller0-objectboxB_-10.0_-10.0_-15.0,3.06602
3,rbohand3/hand0_0-controller1-objectsphere_-10.0_0.0_-15.0,3.1998
4,rbohand3/hand0_0-controller0-objectsphere_-10.0_10.0_15.0,3.21904
5,rbohand3/hand0_0-controller2-objecteggB_-10.0_-10.0_-15.0,3.24273
6,rbohand2/hand0_0-controller1-objectsphere_10.0_5.0_7.5,3.24383
7,rbohand3/hand0_0-controller1-objectsphere_0.0_-10.0_15.0,3.24617
8,rbohand3/hand0_0-controller1-objectsphere_-5.0_-5.0_7.5,3.24847
9,rbohand3/hand0_0-controller1-objectsphere_-10.0_0.0_7.5,3.25897
10,rbohand2/hand0_0-controller1-objecteggB_-5.0_10.0_7.5,3.26514


In [19]:
mccw_files  = map(x->"$x/analysis/mc_cw-averaged_$(wbins)_$(abins).txt", dirs)
mccw_values = map(x->readcsv(x)[1],mccw_files)
names       = []
fd = open("$parent/results_mc_cw-averaged_$(wbins)_$(abins).txt","w")
for i = 1:length(in_dirs)
    v = split(in_dirs[i],"/")
    n = "$(v[8])/$(v[9])"
    append!(names,[n])
    write(fd, "$n,$(mccw_values[i])\n")
end
close(fd)

mccw_results = DataFrame()
mccw_results[:Name] = names
mccw_results[:Values] = mccw_values
sort!(mccw_results, cols=[:Values])
mccw_results[:,:Values] = map(Float64,mccw_results[:,:Values])
writetable("$parent/results_mc_cw-averaged_$(wbins)_$(abins).csv",mccw_results)
mccw_results

,Name,Values
1,rbohand2/hand0_0-controller0-objectbox_10.0_-10.0_-15.0,3.0417
2,rbohand2/hand0_0-controller0-objectboxB_-10.0_-10.0_-15.0,3.06602
3,rbohand3/hand0_0-controller1-objectsphere_-10.0_0.0_-15.0,3.1998
4,rbohand3/hand0_0-controller0-objectsphere_-10.0_10.0_15.0,3.21904
5,rbohand3/hand0_0-controller2-objecteggB_-10.0_-10.0_-15.0,3.24273
6,rbohand2/hand0_0-controller1-objectsphere_10.0_5.0_7.5,3.24383
7,rbohand3/hand0_0-controller1-objectsphere_0.0_-10.0_15.0,3.24617
8,rbohand3/hand0_0-controller1-objectsphere_-5.0_-5.0_7.5,3.24847
9,rbohand3/hand0_0-controller1-objectsphere_-10.0_0.0_7.5,3.25897
10,rbohand2/hand0_0-controller1-objecteggB_-5.0_10.0_7.5,3.26514


In [20]:
results = DataFrame()
sort!(mccw_results, cols=[:Name])
sort!(mcw_results, cols=[:Name])
results[:Name] = mcw_results[:Name]
results[:MC_W] = mcw_results[:Values]
results[:MC_CW] = mccw_results[:Values]
results[:Diff]  = mcw_results[:Values] - mccw_results[:Values]
writetable("$parent/results_mc-averaged_$(wbins)_$(abins).csv",results)
results

,Name,MC_W,MC_CW,Diff
1,rbohand2-prescriptive/hand0_0-controller0-objectboxB_0.0_0.0_0.0,3.87269,3.87269,0.0
2,rbohand2-prescriptive/hand0_0-controller0-objectbox_0.0_0.0_0.0,3.87269,3.87269,0.0
3,rbohand2-prescriptive/hand0_0-controller0-objectcylinderB_0.0_0.0_0.0,3.87269,3.87269,0.0
4,rbohand2-prescriptive/hand0_0-controller0-objectcylinder_0.0_0.0_0.0,3.87269,3.87269,0.0
5,rbohand2-prescriptive/hand0_0-controller0-objecteggB_0.0_0.0_0.0,3.87269,3.87269,0.0
6,rbohand2-prescriptive/hand0_0-controller0-objectegg_0.0_0.0_0.0,3.87269,3.87269,0.0
7,rbohand2-prescriptive/hand0_0-controller0-objectsphereB_0.0_0.0_0.0,3.87269,3.87269,0.0
8,rbohand2-prescriptive/hand0_0-controller0-objectsphere_0.0_0.0_0.0,3.87269,3.87269,0.0
9,rbohand2-prescriptive/hand0_0-controller1-objectboxB_0.0_0.0_0.0,3.70455,3.70455,0.0
10,rbohand2-prescriptive/hand0_0-controller1-objectbox_0.0_0.0_0.0,3.70455,3.70455,0.0


In [21]:
results[results[:Name] .== "rbohand2/hand0_0-controller0-objectboxB_-10.0_-5.0_-15.0", :]

,Name,MC_W,MC_CW,Diff
1,rbohand2/hand0_0-controller0-objectboxB_-10.0_-5.0_-15.0,4.54295,4.54295,0.0


In [22]:
function get_results(exp_name, method)
    values   = []

    for a in [-10.0, -5.0, 0.0, 5.0, 10.0]
        for b in [-10.0, -5.0, 0.0, 5.0, 10.0]
            for c in [-15.0, -7.5, 0.0, 7.5, 15.0]
                append!(values, get_value_by_name(results, "$(exp_name)_$(a)_$(b)_$(c)")[method])
            end
        end
    end
    values = map(Float64,values)
    ename = "$(exp_name)_0.0_0.0_0.0"
#     println("$(exp_name)")
#     println("\\begin{table}[h!]")
#     println("\\begin{center}")
#     println("\\begin{tabular}{ll}")
#     println("  Mean: & $(mean(values))\\\\")
#     println("  STD:  & $(std(values))\\\\")
#     println("  Max:  & $(maximum(values))\\\\")
#     println("  Min:  & $(minimum(values))\\\\")
#     println("  pose = (0,0,0): & $(get_value_by_name(results, "$(ename)")[:MC_W][1])")
#     println("\\end{tabular}")
#     println("\\end{center}")
#     println("% \\caption{}\\label{fig:}")
#     println("\\end{table}")
    return (mean(values), std(values), maximum(values), minimum(values), get_value_by_name(results, "$(ename)")[:MC_W][1])
end

hands        = []
prescriptive = []
controllers  = []
objects      = []
mins         = []
maxs         = []
means        = []
stds         = []
zero_pos     = []
p_values     = []

suffix = ""
for hand in [2,3]
    for controller in [0,1,2]
        for object in ["objectbox", "objectboxB", "objectsphere", "objectsphereB", "objectegg", "objecteggB", "objectcylinder", "objectcylinderB"]
            exp_name ="rbohand$hand$suffix/hand0_0-controller$(controller)-$object"
            (a,b,c,d,e) = get_results(exp_name, :MC_W)
            append!(hands, [hand])
            append!(controllers, [controller])
            append!(objects, [object])
            append!(means,    [a])
            append!(stds,     [b])
            append!(maxs,     [c])
            append!(mins,     [d])
            append!(zero_pos, [e])
        end
    end
end

suffix = "-prescriptive"

for hand in [2,3]
    for controller in [0,1,2]
        for object in ["objectbox", "objectboxB", "objectsphere", "objectsphereB", "objectegg", "objecteggB", "objectcylinder", "objectcylinderB"]
            exp_name ="rbohand$hand$suffix/hand0_0-controller$(controller)-$object"
            (a,b,c,d,e) = get_results(exp_name, :MC_W)
            append!(p_values, [a])
        end
    end
end

In [23]:
object_results                = DataFrame()
object_results[:Hand]         = map(Int64, hands)
object_results[:ControllerID] = map(Int64, controllers)
object_results[:Object]       = objects
object_results[:Mean]         = map(Float64, means)
object_results[:Std]          = map(Float64, stds)
object_results[:Min]          = map(Float64, mins)
object_results[:Max]          = map(Float64, maxs)
object_results[:ZeroPose]     = map(Float64, zero_pos)
object_results[:Prescriptive] = map(Float64, p_values)
writetable("$parent/object_mc_w-averaged_$(wbins)_$(abins).csv",object_results)

object_results[:Mean]         = map(roundF, means)
object_results[:Std]          = map(roundF, stds)
object_results[:Min]          = map(roundF, mins)
object_results[:Max]          = map(roundF, maxs)
object_results[:ZeroPose]     = map(roundF, zero_pos)
object_results[:Prescriptive] = map(roundF, p_values)

writetable("$parent/object_mc_w-averaged_$(wbins)_$(abins)_rounded.csv",object_results)
object_results

,Hand,ControllerID,Object,Mean,Std,Min,Max,ZeroPose,Prescriptive
1,2,0,objectbox,4.421,0.344,3.042,5.072,4.786,3.873
2,2,0,objectboxB,4.116,0.252,3.066,4.671,4.205,3.873
3,2,0,objectsphere,4.151,0.453,3.448,5.017,3.683,3.873
4,2,0,objectsphereB,4.335,0.399,3.288,4.985,4.732,3.873
5,2,0,objectegg,4.208,0.452,3.5,5.072,3.903,3.873
6,2,0,objecteggB,4.429,0.267,3.788,4.95,4.345,3.873
7,2,0,objectcylinder,4.484,0.234,3.668,4.847,4.51,3.873
8,2,0,objectcylinderB,4.471,0.365,3.688,5.043,4.581,3.873
9,2,1,objectbox,4.526,0.318,3.791,5.072,4.184,3.705
10,2,1,objectboxB,4.227,0.249,3.673,4.956,4.227,3.705


In [33]:
#results = readtable("$parent/results_mc-averaged_$(wbins)_$(abins).csv")
#object_results = readtable("$parent/object_mc_w-averaged_$(wbins)_$(abins).csv")

get_max_from_results(hand, controller, object) = map(Float64,object_results[ (object_results[:Hand] .== hand ) & (object_results[:ControllerID] .== controller) & (object_results[:Object] .== object),:][:Max][1])

function findstringarray(ary, s)
   res = Any[]
   for e in ary
    contains(e, s) ? push!(res, true) : push!(res, false)
   end
   return res
end

function get_name_from_results(hand, controller, object)
    m = get_max_from_results(hand, controller, object)
    r = results[find(findstringarray(results[:,:Name],shortname(hand,"",controller,object))), :]
    r = r[(r[:MC_W] .< m + 0.001) & (r[:MC_W] .> m - 0.001),:]
    r[:Name][1]
end

for hand in [2,3]
    for controller in [0,1,2]
        for object in ["objectbox", "objectboxB", "objectsphere", "objectsphereB", "objectegg", "objecteggB", "objectcylinder", "objectcylinderB"]
            println(get_name_from_results(hand,controller,object))
        end
    end
end

rbohand2/hand0_0-controller0-objectbox_0.0_5.0_15.0
rbohand2/hand0_0-controller0-objectboxB_-10.0_10.0_7.5
rbohand2/hand0_0-controller0-objectsphere_0.0_-5.0_-7.5
rbohand2/hand0_0-controller0-objectsphereB_-5.0_-5.0_-15.0
rbohand2/hand0_0-controller0-objectegg_10.0_-5.0_7.5
rbohand2/hand0_0-controller0-objecteggB_-5.0_5.0_-7.5
rbohand2/hand0_0-controller0-objectcylinderB_0.0_5.0_7.5
rbohand2/hand0_0-controller0-objectcylinderB_10.0_5.0_-15.0
rbohand2/hand0_0-controller1-objectbox_-5.0_0.0_-15.0
rbohand2/hand0_0-controller1-objectboxB_5.0_10.0_-7.5
rbohand2/hand0_0-controller1-objectsphereB_-10.0_-5.0_-7.5
rbohand2/hand0_0-controller1-objectsphereB_10.0_-10.0_7.5
rbohand2/hand0_0-controller1-objectegg_5.0_-10.0_-15.0
rbohand2/hand0_0-controller1-objecteggB_10.0_0.0_-7.5
rbohand2/hand0_0-controller1-objectcylinder_-5.0_-10.0_-15.0
rbohand2/hand0_0-controller1-objectcylinderB_0.0_0.0_-7.5
rbohand2/hand0_0-controller2-objectbox_-5.0_5.0_15.0
rbohand2/hand0_0-controller2-objectboxB_10.0_-5.

# Create Videos

In [32]:
experiments = ["rbohand2/hand0_0-controller0-objectbox_0.0_5.0_15.0",
"rbohand2/hand0_0-controller0-objectsphere_10.0_10.0_-15.0",
"rbohand2/hand0_0-controller0-objectcylinder_-10.0_5.0_-15.0",
"rbohand2/hand0_0-controller1-objectbox_-5.0_5.0_-15.0",
"rbohand2/hand0_0-controller1-objectsphere_5.0_-5.0_-15.0",
"rbohand2/hand0_0-controller1-objectcylinder_-5.0_-5.0_-15.0",
"rbohand2/hand0_0-controller2-objectbox_-5.0_5.0_15.0",
"rbohand2/hand0_0-controller2-objectsphere_5.0_-5.0_-7.5",
"rbohand3/hand0_0-controller0-objectbox_5.0_5.0_-15.0",
"rbohand3/hand0_0-controller0-objectsphere_0.0_10.0_-7.5",
"rbohand3/hand0_0-controller0-objectcylinder_-5.0_-5.0_-7.5",
"rbohand3/hand0_0-controller1-objectbox_5.0_-10.0_-7.5",
"rbohand3/hand0_0-controller1-objectsphere_-5.0_5.0_-15.0",
"rbohand3/hand0_0-controller1-objectcylinder_-5.0_-10.0_0.0",
"rbohand3/hand0_0-controller2-objectbox_10.0_10.0_-15.0",
"rbohand3/hand0_0-controller2-objectsphere_0.0_10.0_0.0",
"rbohand3/hand0_0-controller2-objectcylinder_-5.0_5.0_-7.5"]

fd = open("$parent/run.sh","w")

for exps in experiments
    write(fd, "cd $parent\n")
    write(fd, "cd $exps\n")
    write(fd, "cd setup\n")
    n = replace(exps,"/","_")
    write(fd, "export LD_LIBRARY_PATH=/Users/zahedi/projects/3rd_party/sofa/v16.08/build/install/lib\n")
    write(fd, "/Users/zahedi/projects/3rd_party/sofa/v16.08/build/install/bin/runSofa replay.scn\n")
    write(fd, "avconv -r 20 -i  replay_%08d.png -r 20 -c:v libx264 ~/Desktop/$n.mp4\n")
    write(fd, "rm replay_*\n")
    write(fd, "cd $parent\n\n")
end

close(fd)

# Evaluate Grasp Success

In [26]:
function get_avg_dist(experiment,n)
#    hand_data   = readcsv("$parent/$experiment/analysis/hand.sofastates.csv")[end-n+1:end,:]
    hand_data   = get_xyz("$parent/$experiment/raw/hand.sofastates.txt")[end-n+1:end,:]
    object_data = readcsv("$parent/$experiment/analysis/obstacle.sofastates.csv")[end-n+1:end,:]
    s = 0
    for i in 1:n
        # 23, because we have the wrist coordinate (not transformed into local CS)
        # palm = hand_data[i,1+23*3:3+23*3]
        # palm = hand_data[i,1:3]
        palm = hand_data[i,1+18*3:3+18*3] # middle finger root
        object = object_data[i,:]
        dist = sqrt(sum((palm-object).^2))
        s = s + dist
    end
    s = s / n
    s
end

get_avg_dist (generic function with 1 method)

In [27]:
grasp_success = DataFrame()
values = []
names  = []
suffix = ""
for hand in [2,3]
    for controller in [0,1,2]
        for object in ["objectbox", "objectboxB", "objectsphere", "objectsphereB", "objectegg", "objecteggB", "objectcylinder", "objectcylinderB"]
            for a in [-10.0, -5.0, 0.0, 5.0, 10.0]
                for b in [-10.0, -5.0, 0.0, 5.0, 10.0]
                    for c in [-15.0, -7.5, 0.0, 7.5, 15.0]
                        exp_name ="rbohand$hand$suffix/hand0_0-controller$(controller)-$(object)_$(a)_$(b)_$(c)" 
                        append!(names,  [exp_name])
                        append!(values, [get_avg_dist(exp_name,25)])
                    end
                end
            end
        end
    end
end

grasp_success[:Exp]    = names
grasp_success[:Values] = map(x->round(x,2), values)
sort!(grasp_success, cols=[:Values])
writetable("$parent/grasp_success_by_experiment_$(wbins)_$(abins).csv", grasp_success)

In [28]:
grasp_success_vs_mc = DataFrame()
grasp_success_vs_mc[:Exp] = grasp_success[:Exp]
grasp_success_vs_mc[:GraspDist] = grasp_success[:Values]

mcs = []

for i = 1:size(grasp_success_vs_mc)[1]
    exp_name = grasp_success_vs_mc[i,:Exp]
    append!(mcs,results[results[:Name] .== exp_name,:MC_W])
end

grasp_success_vs_mc[:MC_W] = map(x->round(x,2),mcs)
writetable("$parent/grasp_success_by_experiment_vs_mcw_$(wbins)_$(abins).csv", grasp_success_vs_mc)
grasp_success_vs_mc

,Exp,GraspDist,MC_W
1,rbohand3/hand0_0-controller1-objecteggB_-5.0_-5.0_-15.0,21.84,4.35
2,rbohand3/hand0_0-controller1-objecteggB_-10.0_-10.0_15.0,22.27,4.57
3,rbohand3/hand0_0-controller0-objecteggB_-5.0_10.0_-15.0,23.35,4.55
4,rbohand3/hand0_0-controller1-objectsphereB_-10.0_10.0_0.0,23.51,4.13
5,rbohand3/hand0_0-controller1-objecteggB_-10.0_5.0_-15.0,23.62,4.22
6,rbohand3/hand0_0-controller1-objectsphereB_-10.0_5.0_-15.0,23.89,4.6
7,rbohand3/hand0_0-controller1-objecteggB_0.0_-5.0_15.0,24.59,4.22
8,rbohand3/hand0_0-controller1-objecteggB_5.0_-10.0_15.0,24.62,4.63
9,rbohand3/hand0_0-controller1-objecteggB_0.0_0.0_0.0,24.76,4.62
10,rbohand3/hand0_0-controller1-objecteggB_-5.0_5.0_-15.0,24.79,3.91


In [29]:
function findstringarray(ary, s)
   res = Any[]
   for e in ary
    contains(e, s) ? push!(res, true) : push!(res, false)
   end
   return res
end

zero_grasp_success = grasp_success_vs_mc[find(findstringarray(grasp_success_vs_mc[:,:Exp],"_0.0_0.0_0.0")), :]
writetable("$parent/grasp_success_by_experiment_vs_mcw_0.0_0.0_0.0_$(wbins)_$(abins).csv", zero_grasp_success)
zero_grasp_success

,Exp,GraspDist,MC_W
1,rbohand3/hand0_0-controller1-objecteggB_0.0_0.0_0.0,24.76,4.62
2,rbohand3/hand0_0-controller1-objectsphereB_0.0_0.0_0.0,29.87,4.76
3,rbohand2/hand0_0-controller1-objectcylinder_0.0_0.0_0.0,49.76,4.42
4,rbohand3/hand0_0-controller1-objectegg_0.0_0.0_0.0,52.52,4.11
5,rbohand3/hand0_0-controller0-objectsphereB_0.0_0.0_0.0,55.21,4.57
6,rbohand2/hand0_0-controller0-objectcylinder_0.0_0.0_0.0,56.78,4.51
7,rbohand2/hand0_0-controller0-objecteggB_0.0_0.0_0.0,57.94,4.34
8,rbohand3/hand0_0-controller0-objecteggB_0.0_0.0_0.0,58.01,4.45
9,rbohand2/hand0_0-controller1-objecteggB_0.0_0.0_0.0,59.77,4.52
10,rbohand3/hand0_0-controller1-objectsphere_0.0_0.0_0.0,60.59,4.55


In [31]:
fd = open("$parent/run.sh","w")

for exps in zero_grasp_success[:Exp]
    write(fd, "cd $parent\n")
    write(fd, "cd $exps\n")
    write(fd, "cd setup\n")
    n = replace(exps,"/","_")
    write(fd, "export LD_LIBRARY_PATH=/Users/zahedi/projects/3rd_party/sofa/v16.08/build/install/lib\n")
    write(fd, "/Users/zahedi/projects/3rd_party/sofa/v16.08/build/install/bin/runSofa replay.scn\n")
    write(fd, "avconv -r 20 -i  replay_%08d.png -r 20 -c:v libx264 ~/Desktop/$n.mp4\n")
    write(fd, "rm replay_*\n")
    write(fd, "cd $parent\n\n")
end

close(fd)